In [57]:
from time import sleep
from spi_helper import *
from litex import RemoteClient
from ad9174 import Ad9174Settings, Ad9174Init

def getId(r):
    s = ""
    for i in range(64):
        temp = r.read(r.bases.identifier_mem + i * 4)
        if temp == 0:
            break
        s += chr(temp & 0xFF)
    return s

# Connec to FPGA

In [25]:
r = RemoteClient(csr_csv='../build/csr.csv', debug=False, port=1234)
r.open()
getId(r)

'AD9174 + VC707 test 2021-02-20 23:28:08'

# Setup AD9174

In [58]:
settings = Ad9174Settings(json_file='../build/csr.json')
print(settings)
ad = Ad9174Init(r, settings)
ad.init_ad9174(ADC_CLK_DIV=4, USE_PLL=True, M_DIV=2, N_DIV=2, OUT_DIV=2)

----------------
 JESD mode 20
----------------
INTERP_CH: 1  INTERP_MAIN: 1  DSP_CLK_DIV: 16
JESD204BSettings(): 5a 05 00 87 00 1f 00 0f 2f 23 80 00 00 e6 
         DID:  90        BID:   5     ADJCNT:   0        LID:   0 
       PHADJ:   0     ADJDIR:   0          L:   8        SCR:   1 
           F:   1          K:  32          M:   1          N:  16 
          CS:   0         NP:  16  SUBCLASSV:   1          S:   4 
       JESDV:   1         CF:   0         HD:   1       RES1:   0 
        RES2:   0       FCHK: 230 
   [ LINK_DW:  32     FR_CLK:   4 ]
AD917X_NVM_BLR_DONE: 1
PROD_ID: 0x9174
PROD_GRADE: 0  DEV_REVISION: 5
DAC PLL locked: 1
DLL locked: 1
SPI_PAGEINDX: 0b01000001
CAL_STAT: 1
SERDES PLL locked: 1
MODE_NOT_IN_TABLE: 0


In [127]:
wr = ad.ad.wr
rr = ad.ad.rr

# Reset the GTX phy

In [146]:
# r.regs.ctrl_reset.write(1)  # resets ALL clockdomains (HARSH!)
r.regs.control_control.write(0b01)  # resets PHYs and jesd core
print('status: {:03b}'.format(r.regs.control_status.read()))

# bit1: links_enable,  bit0: phys_reset
r.regs.control_control.write(0b00)

status: 000


In [147]:
# bit2: /jsync,  bit1: links_ready,  bit0: all phys initialized
print('status: {:03b}'.format(r.regs.control_status.read()))

status: 001


# GTX testpattern --> AD9174 snapshot

In [142]:
# Transmit a square wave of frequency line_rate / 40
on = 1
for i in range(8):
    getattr(r.regs, 'phy{}_transmitter_tp_on'.format(i)).write(on)

r.regs.phy0_transmitter_tp_on.read()

1

In [149]:
wr('PHY_PRBS_TEST_EN', 0xFF)  # Needed: clock to test module
wr('PHY_PRBS_TEST_CTRL', 0b01)  # rst

print('PHY_SNAPSHOT_DATA:')
for lane in range(8):
    wr('PHY_PRBS_TEST_CTRL', (lane << 4))    
    wr('PHY_DATA_SNAPSHOT_CTRL', 0x01)
    wr('PHY_DATA_SNAPSHOT_CTRL', 0x00)
    val = 0
    for i in range(5):
        val = (val << 8) | rr(0x323 - i)
    bVal = '{:040b}'.format(val)
    desc = ''
    if bVal.find('00111110101100000101') >= 0:
        desc = ','
    elif prbs7_str.find(bVal) >= 0:
        desc = 'PRBS7'
    elif prbs7_str_inv.find(bVal) >= 0:
        desc = '/PRBS7'
    print('{0:}: 0x{1:010x}, 0b{2:}, {3:}'.format(lane, val, bVal, desc))

PHY_SNAPSHOT_DATA:
0: 0xff00000fff, 0b1111111100000000000000000000111111111111, 
1: 0x00000fffff, 0b0000000000000000000011111111111111111111, 
2: 0xfffff00000, 0b1111111111111111111100000000000000000000, 
3: 0xffffe00001, 0b1111111111111111111000000000000000000001, 
4: 0x00001ffffe, 0b0000000000000000000111111111111111111110, 
5: 0x07ffff8000, 0b0000011111111111111111111000000000000000, 
6: 0x77ffeffeef, 0b0111011111111111111011111111111011101111, 
7: 0x000fffff00, 0b0000000000001111111111111111111100000000, 


Lane 4 - 7 seem to show commas sometimes!

`K28.5` = 0011111010 / 1100000101 

# PRBS check

In [1422]:
prbs = 0  # 0: PRBS7, 1: PRBS15, 2: PRBS31
ad.wr('PHY_PRBS_TEST_CTRL', (prbs << 2) | 0b00)
ad.wr('PHY_PRBS_TEST_EN', 0xFF)  # Enable test for all lanes
ad.wr('PHY_PRBS_TEST_CTRL', (prbs << 2) | 0b01)  # rst
ad.wr('PHY_PRBS_TEST_CTRL', (prbs << 2) | 0b00)
ad.wr('PHY_PRBS_TEST_THRESHOLD_LOBITS', 0xFF)  # error threshold

ad.wr('PHY_PRBS_TEST_CTRL', (prbs << 2) | 0b10)  # start test
sleep(0.01)
ad.wr('PHY_PRBS_TEST_CTRL', (prbs << 2) | 0b00)  # stop test

print('PHY_PRBS_TEST_STATUS: {:08b} (high = pass)'.format(ad.rr('PHY_PRBS_TEST_STATUS')))
for lane in range(8):
    ad.wr('PHY_PRBS_TEST_CTRL', (lane << 4) | (prbs << 2) | 0b00)  # read-out lane X
    err_cnt = ad.rr('PHY_PRBS_TEST_ERRCNT_HIBITS') << 16 | \
              ad.rr('PHY_PRBS_TEST_ERRCNT_MIDBITS') << 8 | \
              ad.rr('PHY_PRBS_TEST_ERRCNT_LOBITS')
    print('lane: {}, err_cnt: {:06x}'.format(lane, err_cnt))

PHY_PRBS_TEST_STATUS: 10000000 (high = pass)
lane: 0, err_cnt: 88b5e4
lane: 1, err_cnt: 88b5e4
lane: 2, err_cnt: 88b5e4
lane: 3, err_cnt: 88b5e4
lane: 4, err_cnt: 88b5e4
lane: 5, err_cnt: 88b5e4
lane: 6, err_cnt: 88b5e4
lane: 7, err_cnt: 000000


# Setup for internal PRBS check

In [1420]:
ad.wr(0x240, 0)
ad.wr(0x241, 0)
# ad.wr('CDR_BITINVERSE', 0x00)
ad.wr('LBT_REG_CNTRL_0', 0xFF)  # enable loopback for lane X
halfrate = 0  # 1: enable halfrate mode (keep off!)
ad.wr('LBT_REG_CNTRL_1', (halfrate << 1) | 1)
ad.wr('LBT_REG_CNTRL_1', halfrate << 1)

0

In [338]:
ad.rr(0x201)

0